**מטלת סיכום חלק א**

מגישה: אלישבע אוייר 332515956

קישור לגיטהאב- 

In [10]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.ad.co.il/car?sp261=13891"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [11]:
import requests
from bs4 import BeautifulSoup

def get_ad_urls_from_page(page_index):
    base_url = 'https://www.ad.co.il/car?sp261=13891&rp264=2012,2022&pageindex=' 
    full_url = base_url + str(page_index)
# sp261=13891- chevrolet, rp264=2012,2022- years 2012-2022 (2022 is the max on the website)     
    response = requests.get(full_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        ad_links = soup.find_all('a', href=True)
        
        ad_urls = []
        for link in ad_links:
            href = link['href']
            if '/ad/' in href:
                ad_url = 'https://www.ad.co.il' + href
                # had to add a code to check if title = chevrolet
                ad_response = requests.get(ad_url)
                if ad_response.status_code == 200:
                    ad_soup = BeautifulSoup(ad_response.content, 'html.parser')
                    title = ad_soup.find('h2', class_='card-title')
                    if title and 'שברולט' in title.get_text():
                        ad_urls.append(ad_url)
        return ad_urls

# Loop through page indexes and collect relevant ad URLs (3 indexes)
all_ad_urls = []
for page_index in range(1, 4): 
    ad_urls = get_ad_urls_from_page(page_index)
    if ad_urls:
        all_ad_urls.extend(ad_urls)

In [15]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import pandas as pd
import re

def parse_ad_page(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    # Manufacturer
    manufacturer = 'שברולט'

    # Model
    title_tag = soup.find('h2', class_='card-title')
    model = title_tag.text.strip().replace(manufacturer, '').strip()

    # Year
    year_tag = soup.find('td', string=re.compile(r'שנה'))
    year = year_tag.find_next('td').text.strip() if year_tag else None
    
    # Hand
    hand_tag = soup.find('span', string=re.compile('יד \d'))
    hand = int(hand_tag.text.strip().replace('יד ', '')) if hand_tag else None

    # Gearbox
    gearbox_tag = soup.find('td', string=re.compile(r'ת\. הילוכים'))
    gearbox = gearbox_tag.find_next('td').text.strip() if gearbox_tag else None

    # Engine Volume
    engine_volume_tag = soup.find('td', string=re.compile(r'נפח'))
    engine_volume = engine_volume_tag.find_next('td').text.strip() if engine_volume_tag else None

    # Engine Type
    engine_type_tag = soup.find('td', string=re.compile(r'סוג מנוע'))
    engine_type = engine_type_tag.find_next('td').text.strip() if engine_type_tag else None

    # Previous Ownership
    previous_ownership_tag = soup.find('td', string=re.compile(r'בעלות קודמת'))
    previous_ownership = previous_ownership_tag.find_next('td').text.strip() if previous_ownership_tag else None

    # Current Ownership
    current_ownership_tag = soup.find('td', string=re.compile(r'בעלות נוכחית'))
    current_ownership = current_ownership_tag.find_next('td').text.strip() if current_ownership_tag else None

    # Area
    area_tag = soup.find('td', string=re.compile(r'אזור'))
    area = area_tag.find_next('td').text.strip() if area_tag else None

    # City
    city_tag = soup.find('td', string=re.compile(r'עיר'))
    city = city_tag.find_next('td').text.strip() if city_tag else None

   # Price
    price_tag = soup.find('h2', class_='card-title')
    price_text = price_tag.find_next('h2', class_='card-title').text.strip()
    price_match = re.search(r'(\d{1,3}(?:,\d{3})*)(?:\.\d+)? ₪', price_text)
    price = float(price_match.group(1).replace(',', '')) if price_match else None

    # Number of Images - counts both images and video
    image_tags = soup.find_all('img', class_='desktop-thumbnail')  
    num_car_images = len(image_tags)

    # Creation Date
    creation_date_text = soup.find('div', class_='px-3', string=re.compile('תאריך יצירה:'))
    creation_date = datetime.strptime(creation_date_text.text.strip().replace('תאריך יצירה:', '').strip(), '%d/%m/%Y').date() if creation_date_text else None

    # Bump Date
    bump_date_text = soup.find('div', class_='px-3', string=re.compile('תאריך הקפצה אחרון:'))
    bump_date = datetime.strptime(bump_date_text.text.strip().replace('תאריך הקפצה אחרון:', '').strip(), '%d/%m/%Y').date() if bump_date_text else None

    # Description
    description_tag = soup.find('p', class_='text-word-break')
    description = description_tag.text.strip().replace('\n', '').replace('\r', '') if description_tag else None
    description = re.sub(r'[^\w\s\d\.]', '', description) if description else None
  
    # Color
    color_tag = soup.find('td', string=re.compile(r'צבע'))
    color = color_tag.find_next('td').text.strip() if color_tag else None

    # Mileage
    mileage_tag = soup.find('td', string=re.compile(r'ק"מ'))
    mileage_text = mileage_tag.find_next('td').text.strip() if mileage_tag else None
    mileage = int(re.sub(r'\D', '', mileage_text)) if mileage_text else None

    # Test Until
    def calculate_days_until_test(soup):
        test_date_tag = soup.find('td', string=re.compile(r'טסט עד'))
        test_date_text = test_date_tag.find_next('td').text.strip() if test_date_tag else None

        if test_date_text:
            match = re.search(r'(\d{2})/(\d{4})', test_date_text)
            if match:
                month, year = match.groups()
                month = int(month)
                year = int(year)

                # Calculate the last day of the month
                next_month = datetime(year, month % 12 + 1, 1)
                last_day_of_month = next_month - timedelta(days=1)

                # Calculate the number of days until the test date
                today = datetime.now()
                days_until_test = (last_day_of_month - today).days

                return days_until_test

        return None

    # Use the function to calculate days until the next car test
    days_until_test = calculate_days_until_test(soup)

    return {
        'יצרן': manufacturer,
        'דגם': model,
        'שנה': year,
        'יד': hand,
        'ת. הילוכים': gearbox,
        'נפח': engine_volume,
        'סוג מנוע': engine_type,
        'בעלות קודמת': previous_ownership,
        'בעלות נוכחית': current_ownership,
        'אזור': area,
        'עיר': city,
        'מחיר': price,
        'מספר תמונות': num_car_images,
        'תאריך יצירה': creation_date,
        'תאריך הקפצה': bump_date,
        'תיאור': description,
        'צבע': color,
        'ק"מ': mileage,
        'טסט עד': days_until_test, 
    }


In [16]:
# Scrape all the Chevrolet ad pages
all_ads_data = []
for ad_url in all_ad_urls:
    ad_data = parse_ad_page(ad_url)
    all_ads_data.append(ad_data)

# Convert to DataFrame for easier handling
ads_df = pd.DataFrame(all_ads_data)
ads_df.to_csv('file.csv',encoding='utf-8-sig')

display(ads_df)


,יצרן,דגם,שנה,יד,ת. הילוכים,נפח,סוג מנוע,בעלות קודמת,בעלות נוכחית,אזור,עיר,מחיר,מספר תמונות,תאריך יצירה,תאריך הקפצה,תיאור,צבע,"ק""מ",טסט עד
0,שברולט,קרוז,2013,2,אוטומטית,"1,600",בנזין,ליסינג,פרטית,רמת גן - גבעתיים,רמת גן,9990.0,6,2024-02-28,2024-06-09,מודל 2013 אוטומתי יד שנייה מטופלת נעילה מרכזית...,אפור עכבר,208000.0,169.0
1,שברולט,אורלנדו,2017,2,אוטומטית,"1,400",בנזין,השכרה,פרטית,גליל ועמקים,בית רימון,31000.0,3,2024-05-20,2024-05-27,שמור ומטופל בזמן במוסך החברה שברולט טבריה.לאחר...,שחור,165000.0,NaN
2,שברולט,ספארק,2020,1,אוטומטית,"1,500",בנזין,פרטית,פרטית,טבריה והסביבה,מגאר,70000.0,4,2024-03-28,2024-03-28,רכב שמור דגם הכי מפואר,לבן מטאלי,47000.0,351.0
3,שברולט,קרוז,2013,2,אוטומטית,"1,600",בנזין,פרטית,פרטית,פתח תקוה והסביבה,פתח תקווה,19800.0,4,2024-03-11,2024-03-12,יד2 פרטי במצב מכני מעולה,אפור מטאלי,63000.0,NaN
4,שברולט,אקווינוקס,2022,1,אוטומטית,"1,500",בנזין,פרטית,פרטית,ראש העין והסביבה,כפר קאסם,130000.0,4,2024-03-11,2024-03-10,טרייל בליזר הרכב שמור יפה 42000 קמ טיפולים בז...,אפור מטאלי,42000.0,259.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,שברולט,אקווינוקס,2018,3,אוטומטית,"1,490",בנזין,None,None,None,פורת,111000.0,0,2022-06-21,2022-06-21,שברולט אקווינקס 2018 טורבו מצב מכני מעולה באחר...,None,73400.0,NaN
116,שברולט,ספארק,2016,1,אוטומטית,"1,400",בנזין,פרטית,פרטית,None,בית ג'ן,1.0,5,2021-12-10,2021-12-10,יצרן שברולט דגם ספארק נפח 1400הספק כס98 ש...,לבן,111000.0,NaN
117,שברולט,מאליבו,2015,3,אוטומטית,"2,000",בנזין,None,None,None,שגב שלום,42000.0,0,2022-06-05,2022-06-05,רכב מעולה ltbrgtltbrgtפירוטltbrgt קול הצפות,None,160.0,NaN
118,שברולט,ספארק,2015,1,ידנית,"1,200",בנזין,None,None,תל אביב,תל אביב יפו,NaN,0,2022-05-21,2022-05-21,None,None,105000.0,NaN
